# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [159]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [160]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [161]:
# frequency table for prestige and whether or not someone was admitted
freq_prestige = pd.crosstab(index = df["prestige"], columns = "count")
freq_prestige

col_0,count
prestige,
1.0,61
2.0,148
3.0,121
4.0,67


In [162]:
freq_admit = pd.crosstab(index = df["admit"], columns = "count")
freq_admit

col_0,count
admit,
0,271
1,126


In [163]:
prestigeadmit = pd.crosstab(index = df["prestige"], columns = df["admit"])
prestigeadmit

admit,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


In [164]:
prestigeadmit2 = pd.crosstab(index = df["admit"], columns = df["prestige"])
prestigeadmit2.index = ["not admitted", "admitted"]
prestigeadmit2

prestige,1.0,2.0,3.0,4.0
not admitted,28,95,93,55
admitted,33,53,28,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [165]:
dummy_ranks = pd.get_dummies(df, columns = ["prestige"], drop_first = False)

In [166]:
dummy_ranks.head()

,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: We only need three dummy variables for the four prestige levels, but I left four in because it would be easier to visualize odds in the following questions.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [167]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2.0':])
print handCalc.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


In [168]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
cols = [1.0]
prestige1 = prestigeadmit2[cols]
prestige1

prestige,1.0
not admitted,28
admitted,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [169]:
admittedodds1 = (prestige1.ix["admitted", :])/(prestige1.ix["not admitted", :])
admittedodds1

prestige
1.0    1.178571
dtype: float64

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [170]:
cols_new = [2.0, 3.0, 4.0]
prestigeother = prestigeadmit2[cols_new]
prestigeother

prestige,2.0,3.0,4.0
not admitted,95,93,55
admitted,53,28,12


In [171]:
admittedodds = (prestigeother.ix["admitted", :])/(prestigeother.ix["not admitted", :])
admittedodds

prestige
2.0    0.557895
3.0    0.301075
4.0    0.218182
dtype: float64

In [172]:
admitted = prestigeother.sum(axis = 1)
admitted

not admitted    243
admitted         93
dtype: int64

#### 3.3 Calculate the odds ratio

In [173]:
odds = 93.0/243
odds


0.38271604938271603

#### 3.4 Write this finding in a sentenance: 

Answer: Someone who went to a school that is not of prestige 1 has a 0.38 to 1 chance of being admitted.

#### 3.5 Print the cross tab for prestige_4

In [174]:
cols4 = [4.0]
prestige4 = prestigeadmit2[cols4]
prestige4

prestige,4.0
not admitted,55
admitted,12


#### 3.6 Calculate the OR 

In [175]:
admittedodds4 = (prestige4.ix["admitted", :])/(prestige4.ix["not admitted", :])
admittedodds4

prestige
4.0    0.218182
dtype: float64

#### 3.7 Write this finding in a sentence

Answer: Someone who went to a school of Prestige 4 has a 0.21 to 1 chance of being admitted.

## Part 4. Analysis

In [176]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2.0':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [177]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [178]:
train_cols = ["gre", "gpa", "prestige_2.0", "prestige_3.0", "prestige_4.0"]


#### 4.2 Fit the model

In [179]:
from sklearn.linear_model import LogisticRegression

In [180]:
result = LogisticRegression().fit(data[train_cols], data["admit"])
result

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#### 4.3 Print the summary results

In [181]:
print lm.coef_
print lm.intercept_
print data.admit.mean()

[[ 0.00178497  0.23229458 -0.60347467 -1.17214957 -1.37729795]]
[-1.81701706]
0.317380352645


#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [211]:
for i in lm.coef_:
    print i/(1-i)

[ 0.00178816  0.30258296 -0.37635435 -0.53962655 -0.57935437]


In [183]:
params = result.params
conf["OR"]=params
conf.columns = ["2.5%", "97.5%", "OR"]

AttributeError: 'LogisticRegression' object has no attribute 'params'

In [184]:
result1 = sm.Logit(data["admit"], data[train_cols]).fit()

Optimization terminated successfully.
         Current function value: 0.589121
         Iterations 5


In [185]:
print result1.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      392
Method:                           MLE   Df Model:                            4
Date:                Tue, 23 May 2017   Pseudo R-squ.:                 0.05722
Time:                        07:21:41   Log-Likelihood:                -233.88
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.039e-05
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0014      0.001      1.308      0.191        -0.001     0.003
gpa             -0.1323      0.195     -0.680      0.497        -0.514     0.249
prestige_2.0    -0.9562      0.302     -3.17

In [212]:
odds = np.exp(lm.coef_)
odds

array([[ 1.00178657,  1.26149128,  0.546908  ,  0.3097005 ,  0.25225925]])

In [186]:
params =result1.params
conf = result1.conf_int()
conf["OR"] = params
conf.columns = ["2.5%", "97.5", "OR"]
print np.exp(conf)

                  2.5%      97.5        OR
gre           0.999320  1.003420  1.001368
gpa           0.598303  1.282800  0.876073
prestige_2.0  0.212826  0.694082  0.384342
prestige_3.0  0.112055  0.412207  0.214918
prestige_4.0  0.070176  0.338540  0.154135


#### 4.5 Interpret the OR of Prestige_2

Answer: Someone who applies from a school of prestige 2 has a 0.55 to 1 chance of being admitted.

#### 4.6 Interpret the OR of GPA

Answer: Every increment of 1 that a person's GPA increases, their odds of being admitted increase by 1.26 to 1.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [187]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [188]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [189]:
combos.head()

,0,1,2,3
0,220.0,2.260000,1.0,1.0
1,220.0,2.260000,2.0,1.0
2,220.0,2.260000,3.0,1.0
3,220.0,2.260000,4.0,1.0
4,220.0,2.453333,1.0,1.0


In [190]:
# recreate the dummy variables
new_dummies = ["prestige_2.0", "prestige_3.0", "prestige_4.0"]
# keep only what we need for making predictions
newcols = [0, 1]
data2 = combos[newcols].join(dummy_ranks.ix[:, "prestige_2.0":])
data2.head()

,0,1,prestige_2.0,prestige_3.0,prestige_4.0
0,220.0,2.260000,0.0,1.0,0.0
1,220.0,2.260000,0.0,1.0,0.0
2,220.0,2.260000,0.0,0.0,0.0
3,220.0,2.260000,0.0,0.0,1.0
4,220.0,2.453333,0.0,0.0,1.0


#### 5.2 Make predictions on the enumerated dataset

In [195]:
newdata = data2.dropna()

In [197]:
newdata.head()

,0,1,prestige_2.0,prestige_3.0,prestige_4.0
0,220.0,2.260000,0.0,1.0,0.0
1,220.0,2.260000,0.0,1.0,0.0
2,220.0,2.260000,0.0,0.0,0.0
3,220.0,2.260000,0.0,0.0,1.0
4,220.0,2.453333,0.0,0.0,1.0


In [208]:
column = newdata[[0, 1, "prestige_2.0", "prestige_3.0", "prestige_4.0"]]
true = lm.predict(column)
probs = lm.predict_proba(column)
print true[:4]
print probs[:4] 

[0 0 0 0]
[[ 0.88810083  0.11189917]
 [ 0.88810083  0.11189917]
 [ 0.71081335  0.28918665]
 [ 0.90692343  0.09307657]]


#### 5.3 Interpret findings for the last 4 observations

Answer: The probability of the last 4 observations happening is very low (at least under 28.9% for each observation).

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.